### Manipulating the training data on the fly
What is real cool in Lasagne is the ability to easily manipulate the training (and test-data) on the fly. This often done in a random fashion by applying transformations under which the objects are invariant. All winners of competitions do a **clever** data augmentation. So let's give it a try... 

#### BatchIterator the point of entry
To do so one has to overwrite a BatchIterator and implement our own transform method. In the following, we simply flip the image on the y-axis, but in principle quite complex operations using cv2 or other libraries are possible.

#### Flipping
For the non-python freaks as me a short reminder that flipping can be done as follows:

In [1]:
a = (1,2,3,4)
a[::-1]

(4, 3, 2, 1)

#### Loading the data
We load the net and data from the last session via the import.

In [2]:
#We load the net and the data from the last example
from SimpleNet import net1,X,y 
X.shape

Couldn't import dot_parser, loading of dot files will not be possible.
Loaded data
Shape of X (70000, 784)
  Min / Max X 0.0 0.996094
Shape of Y (70000,)
  Min / Max Y 0 9
After Batchnormalization Min / Max X / Mean -1.2488 / 66.487 / 8.03533e-10
Loaded pixels


Using gpu device 1: GeForce GTX 780


(70000, 1, 28, 28)

Fitting the model, with the standard BatchIterator which does not do on the fly manipulations.

In [3]:
d = net1.fit(X[0:1000,:,:,:],y[0:1000]); #Training with the 

  InputLayer        	(None, 1, 28, 28)   	produces     784 outputs
  Conv2DLayer       	(None, 32, 26, 26)  	produces   21632 outputs
  MaxPool2DLayer    	(None, 32, 13, 13)  	produces    5408 outputs
  Conv2DLayer       	(None, 64, 12, 12)  	produces    9216 outputs
  MaxPool2DLayer    	(None, 64, 6, 6)    	produces    2304 outputs
  DenseLayer        	(None, 500)         	produces     500 outputs
  DenseLayer        	(None, 10)          	produces      10 outputs

 Epoch  |  Train loss  |  Valid loss  |  Train / Val  |  Valid acc  |  Dur
--------|--------------|--------------|---------------|-------------|-------
     1  |    2.254012  |    2.129219  |     1.058610  |     40.93%  |  0.2s
     2  |    1.936433  |    1.697138  |     1.140999  |     58.33%  |  0.2s
     3  |    1.331079  |    1.016569  |     1.309384  |     73.01%  |  0.2s
     4  |    0.757213  |    0.717663  |     1.055110  |     81.93%  |  0.2s
     5  |    0.477344  |    0.679934  |     0.702045  |     80.61%  |  0.2

#### Custom BatchIterator
Now we create our own BatchIterator which does the flipping around the x-axis.

In [4]:
from nolearn.lasagne import BatchIterator

class SimpleBatchIterator(BatchIterator):

    def transform(self, Xb, yb):
        # The 'incomming' and outcomming shape is (10, 1, 28, 28)
        Xb, yb = super(SimpleBatchIterator, self).transform(Xb, yb)
        return Xb[:,:,:,::-1], yb #<--- Here we do the flipping of the images

# Setting the new batch iterator
net1.batch_iterator_train = SimpleBatchIterator(batch_size=10)
d = net1.fit(X[0:1000,:,:,:],y[0:1000])


  InputLayer        	(None, 1, 28, 28)   	produces     784 outputs
  Conv2DLayer       	(None, 32, 26, 26)  	produces   21632 outputs
  MaxPool2DLayer    	(None, 32, 13, 13)  	produces    5408 outputs
  Conv2DLayer       	(None, 64, 12, 12)  	produces    9216 outputs
  MaxPool2DLayer    	(None, 64, 6, 6)    	produces    2304 outputs
  DenseLayer        	(None, 500)         	produces     500 outputs
  DenseLayer        	(None, 10)          	produces      10 outputs

 Epoch  |  Train loss  |  Valid loss  |  Train / Val  |  Valid acc  |  Dur
--------|--------------|--------------|---------------|-------------|-------
     1  |    1.220050  |    1.467812  |     0.831203  |     54.71%  |  0.4s
     2  |    0.286907  |    1.958746  |     0.146475  |     49.61%  |  0.4s
     3  |    0.127747  |    2.676057  |     0.047737  |     40.71%  |  0.4s
     4  |    0.072940  |    3.292909  |     0.022151  |     40.19%  |  0.4s
     5  |    0.033829  |    3.380977  |     0.010006  |     46.22%  |  0.4

We also see that it takes a bit longer (0.4 sec), which is strange since in theory the flipping should be done on the CPU while the GPU is buisy doing the fitting. Maybe the network is not big enough to keep the GPU buissy? Anyway.

The second observation is that the performance drops from approx 80% to approx 40%. The reason is that maybe its not such a good idea to flip the training data in the first place. Any idea why?

In [6]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as imgplot
fig = plt.figure()
for i in range(10):
    a=fig.add_subplot(1,20,2*i+1,xticks=[], yticks=[])
    plt.imshow(X[i,0,:,:], cmap=plt.get_cmap('gray'))
    a=fig.add_subplot(1,20,2*i+2,xticks=[], yticks=[])
    plt.imshow(X[i,0,:,::-1], cmap=plt.get_cmap('gray'))    

**Maybe we should do a more clever data augmentation next time...** At least we have the toolbox ready.

If overwritting the `transform` method is not sufficient, one might even overwrite the `__iter__` method in the BatchIterator.